In [1]:
from langchain.text_splitter import CharacterTextSplitter
from dotenv import load_dotenv
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import FAISS
from langchain.chains.question_answering import load_qa_chain
from langchain.llms import OpenAI
from langchain.callbacks import get_openai_callback

In [2]:
from IPython.display import JSON

In [3]:
text = """This is a text extract from https://www.homesmartmesh.com/

it consists of the first section 'Netowrks' but 'Frameworks, Applications, Assistans' have been omitted

# Networks
Ethernet and wifi being used by most generic consumer devices is very convenient and is used wherever it is possible. Even smaller devices with relatively low power such as ESP32 can use wifi, which solves all communication questions and can even provide a webserver hosted on the microcontroller. Usage of websocket allow browsers and webapps such as Home 3D to be independent from proxy servers which increases the flexibility of the client servers architecture and allows clients to run on any device (Smartphone / PC : mac, win,linux). MQTT lowers the application overhead in comparison to REST API and solves all communication scenarios with client polling and server based events notification.
"""

splitter = CharacterTextSplitter(
separator="\n",
chunk_size=300,
chunk_overlap=50,
length_function=len
)
chunks = splitter.split_text(text)

In [18]:
import json
print(json.dumps(chunks,indent=4))

[
    "This is a text extract from https://www.homesmartmesh.com/\nit consists of the first section 'Netowrks' but 'Frameworks, Applications, Assistans' have been omitted\n# Networks",
    "Ethernet and wifi being used by most generic consumer devices is very convenient and is used wherever it is possible. Even smaller devices with relatively low power such as ESP32 can use wifi, which solves all communication questions and can even provide a webserver hosted on the microcontroller. Usage of websocket allow browsers and webapps such as Home 3D to be independent from proxy servers which increases the flexibility of the client servers architecture and allows clients to run on any device (Smartphone / PC : mac, win,linux). MQTT lowers the application overhead in comparison to REST API and solves all communication scenarios with client polling and server based events notification."
]


In [5]:
load_dotenv()
embeddings = OpenAIEmbeddings()
knowledge_base = FAISS.from_texts(chunks, embeddings)

In [7]:
print(knowledge_base)

In [8]:
question = "How can MQTT help the consumer network ?"
references = knowledge_base.similarity_search(question)

In [25]:
JSON(references)

<IPython.core.display.JSON object>

In [11]:
llm = OpenAI()
chain = load_qa_chain(llm, chain_type="stuff")
with get_openai_callback() as cb:
    response = chain.run(input_documents=references, question=question)
    print(cb)

Tokens Used: 275
	Prompt Tokens: 239
	Completion Tokens: 36
Successful Requests: 1
Total Cost (USD): $0.0055


In [12]:
print(response)

 MQTT can help the consumer network by lowering the application overhead in comparison to REST API, and it can also solve all communication scenarios with client polling and server based events notification.
